In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
import transformers
from transformers import AutoModel
import sys
import math
sys.path.insert(0,'..')

from utils.preprocessing import load_data
from utils.transformer_dataset import ReviewDataset
from utils.training import train_text_model, train_text_meta_model, train_meta_model
from models.transformer_models import UsefulScoreRegressorTextOnly, UsefulScoreRegressorAllFeat, UsefulScoreRegressorMetaOnly

In [3]:
# Load data
train, val = load_data('../data/drugsComTrain_raw.csv')

In [ ]:
# See dataframe
train

#### Develop BERT Model

In [4]:
##### Create pytorch dataset
nonTextCols = ['ratingNormalized', 'ageScore', 'ADHD', 'Acne', 'Anxiety', 'Bipolar Disorde',
                'Birth Control', 'Depression', 'Insomnia', 'Obesity', 'Pain', 'Weight Loss']
targetCol = 'usefulScore'

trainset = ReviewDataset(train, 'roberta-base', nonTextCols, targetCol)
valset = ReviewDataset(val, 'roberta-base', nonTextCols, targetCol)
train_loader = DataLoader(dataset=trainset, batch_size=8, shuffle=True)
val_loader = DataLoader(dataset=valset, batch_size=8, shuffle=False)

#### Train Models (Frozen Transformer Weights)

#### Text-only Model

In [5]:
##### Text-only Transformer Model
encoder = AutoModel.from_pretrained('roberta-base', return_dict=True)

# Freeze encoder parameters to avoid CUDA out of memory.
for param in encoder.parameters():
    param.requires_grad = False

model = UsefulScoreRegressorTextOnly(encoder)
model = model.cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

train_text_model(num_epochs=3, model=model, optimizer=optimizer,
                 train_loader=train_loader, val_loader=val_loader,
                 criterion=criterion, save_path='../models/RoBERTa_Frozen_TextOnly_Clip.pt', clip=1.0)

Epoch 0, val loss: inf -> 0.00274, train loss: 0.00334
Epoch 1, val loss: 0.00274 -> 0.00273, train loss: 0.00307
Epoch 2, val loss: 0.00293, train loss: 0.00305


#### Text + Metadata model

In [6]:
##### Text + metadata Transformer Model
encoder = AutoModel.from_pretrained('roberta-base', return_dict=True)

# Freeze encoder parameters to avoid CUDA out of memory.
for param in encoder.parameters():
    param.requires_grad = False

model = UsefulScoreRegressorAllFeat(encoder, num_meta_feats=len(nonTextCols))
model = model.cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

train_text_meta_model(num_epochs=3, model=model, optimizer=optimizer,
                      train_loader=train_loader, val_loader=val_loader,
                      criterion=criterion, save_path='../models/RoBERTa_Frozen_TextMeta_Clip.pt', clip=1.0)

Epoch 0, val loss: inf -> 0.00220, train loss: 0.00268
Epoch 1, val loss: 0.00220 -> 0.00214, train loss: 0.00242
Epoch 2, val loss: 0.00214 -> 0.00213, train loss: 0.00236


In [7]:
#### Check how the model is performing across each metadata feature group
#### Especially important to look at performance by age of review, to see if performance is good for young reviews
#### In practice, the newly posted reviews would be the ones that the model would help prioritize.

#### Train Models (Free Transformer Weights)

#### Text-only Model

In [8]:
##### Text-only Transformer Model
encoder = AutoModel.from_pretrained('roberta-base', return_dict=True)

model = UsefulScoreRegressorTextOnly(encoder)
model = model.cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

train_text_model(num_epochs=3, model=model, optimizer=optimizer,
                 train_loader=train_loader, val_loader=val_loader,
                 criterion=criterion, save_path='../models/RoBERTa_Free_TextOnly_Clip.pt', clip=1.0)

RuntimeError: CUDA out of memory. Tried to allocate 12.00 MiB (GPU 0; 11.00 GiB total capacity; 9.52 GiB already allocated; 8.50 MiB free; 9.61 GiB reserved in total by PyTorch)

#### Text + Metadata Model

In [ ]:
##### Text + metadata Transformer Model
encoder = AutoModel.from_pretrained('roberta-base', return_dict=True)

model = UsefulScoreRegressorAllFeat(encoder, num_meta_feats=len(nonTextCols))
model = model.cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

train_text_meta_model(num_epochs=3, model=model, optimizer=optimizer,
                      train_loader=train_loader, val_loader=val_loader,
                      criterion=criterion, save_path='../models/RoBERTa_Free_TextMeta_Clip.pt', clip=1.0)

#### Train Meta-only Model

#### With Gradient Clipping

In [9]:
model = UsefulScoreRegressorMetaOnly(num_meta_feats=len(nonTextCols))
model = model.cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

train_meta_model(num_epochs=3, model=model, optimizer=optimizer,
                 train_loader=train_loader, val_loader=val_loader,
                 criterion=criterion, save_path='../models/RoBERTa_MetaOnly_Clip.pt', clip=1.0)

Epoch 0, val loss: inf -> 0.00221, train loss: 0.00271
Epoch 1, val loss: 0.00221 -> 0.00218, train loss: 0.00231
Epoch 2, val loss: 0.00218 -> 0.00218, train loss: 0.00227


#### Without Gradient Clipping

In [10]:
model = UsefulScoreRegressorMetaOnly(num_meta_feats=len(nonTextCols))
model = model.cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

train_meta_model(num_epochs=3, model=model, optimizer=optimizer,
                 train_loader=train_loader, val_loader=val_loader,
                 criterion=criterion, save_path='../models/RoBERTa_MetaOnly_NoClip.pt', clip=1000.0)

Epoch 0, val loss: inf -> 0.00224, train loss: 0.00244
Epoch 1, val loss: 0.00224 -> 0.00220, train loss: 0.00230
Epoch 2, val loss: 0.00220, train loss: 0.00228
